#  CV3DST  ReID
- to train a small ReID dataset with cross-entropy and triplet-loss.

#### Install and import Python libraries

In [ ]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

import motmetrics as mm

%load_ext autoreload
%autoreload 2
%matplotlib inline



mm.lap.default_solver = "lap"

In [ ]:
reid_root_dir = ".."
root_dir = ".."
sys.path.append(os.path.join(reid_root_dir, "src"))

In [ ]:
# Load helper code
from market import metrics, utils
from market.datamanager import ImageDataManager
from market.eval import evaluate
from market.models import build_model
from mot.tracker.base import Tracker


In [ ]:
seed = 12345
seq_name = "MOT16-reid"  # We recommend to use this subset.
data_dir = os.path.join(root_dir, "data/MOT16")
output_dir = os.path.join(root_dir, "output")

## Setup

In [ ]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# Training a ReID Network

train a simple ReID network on the Market data. we will use a ResNet34/ResNet50 neural network that extracts features from an input image. 

Next, create the the DataManager for the Market dataset that will provide the train and test sets:

In [ ]:
datamanager = ImageDataManager(
    root=reid_root_dir,
    height=256,
    width=128,
    batch_size_train=32,
    workers=2,
    transforms=["random_flip", "random_crop"],
)
train_loader = datamanager.train_loader
test_loader = datamanager.test_loader

Now, let's create a resnet34 model and move it to the GPU.

In [ ]:
model = build_model(
    "resnet34", datamanager.num_train_pids, loss="softmax", pretrained=True
)
model = model.cuda()

trainable_params = model.parameters()

For training the network, we now need to choose an optimizer and learning rate scheduler.

In [ ]:
optimizer = torch.optim.Adam(
    trainable_params, lr=0.0003, weight_decay=5e-4, amsgrad=True
)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)

The network will be trained on a cross-entropy loss, i.e., the network needs to classify each image to it's identity class. For $n$ different people, we will have $n$ different classes.

During evaluation, we ignore the last classification layer and work on the extracted $feat$-dimensional features. This feature vector should be very similar for the same instance, and not similar for different instances.

In the following, you have to implement two distance measurements:
- Euclidian squared distance.
- Cosine similarity.

You are not allowed to change the interface of the function. Please have a look at the [Pytorch documentation](https://pytorch.org/docs/stable/index.html).

In [ ]:
from mot.utils import cosine_distance, euclidean_squared_distance

With the implemented distance measure, we can now implement the evaluation function. We extract features for the query set and for the gallery set and then build a distance matrix based on your implemented distance measure.
Select metric_fn one of:

- cosine_distance
- euclidean_squared_distance

In [ ]:
metric_fn = cosine_distance  # cosine_distance or euclidean_squared_distance

Finally, we can implement the training logic.

In [ ]:
MAX_EPOCH = 30
EPOCH_EVAL_FREQ = 5
PRINT_FREQ = 50

num_batches = len(train_loader)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(MAX_EPOCH):
    losses = utils.MetricMeter()
    batch_time = utils.AverageMeter()
    end = time.time()
    model.train()
    for batch_idx, data in enumerate(train_loader):
        # Predict output.
        imgs, pids = data["img"].cuda(), data["pid"].cuda()
        output = model(imgs)
        # Compute loss.
        loss = criterion(output, pids)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        losses.update({"Loss": loss})
        if (batch_idx + 1) % PRINT_FREQ == 0:
            utils.print_statistics(
                batch_idx, num_batches, epoch, MAX_EPOCH, batch_time, losses
            )
        end = time.time()

    if (epoch + 1) % EPOCH_EVAL_FREQ == 0 or epoch == MAX_EPOCH - 1:
        rank1, mAP = evaluate(model, test_loader)
        print(
            "Epoch {0}/{1}: Rank1: {rank}, mAP: {map}".format(
                epoch + 1, MAX_EPOCH, rank=rank1, map=mAP
            )
        )

# Part II - Triplet loss and hard negative mining.

Now, we can combine both losses and train a new model.

In [ ]:
from mot.models.reid_losses import CombinedLoss, HardBatchMiningTripletLoss

In [ ]:
model = build_model(
    "resnet34", datamanager.num_train_pids, loss="triplet", pretrained=True
)
model = model.cuda()

trainable_params = model.parameters()
optimizer = torch.optim.Adam(
    trainable_params, lr=0.0003, weight_decay=5e-4, amsgrad=True
)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)

In [ ]:
MAX_EPOCH = 30
EPOCH_EVAL_FREQ = 5
PRINT_FREQ = 10

num_batches = len(train_loader)
criterion = CombinedLoss(0.3, 1.0, 1.0)

for epoch in range(MAX_EPOCH):
    losses = utils.MetricMeter()
    batch_time = utils.AverageMeter()
    end = time.time()
    model.train()
    for batch_idx, data in enumerate(train_loader):
        # Predict output.
        imgs, pids = data["img"].cuda(), data["pid"].cuda()
        logits, features = model(imgs)
        # Compute loss.
        loss, loss_summary = criterion(logits, features, pids)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        losses.update(loss_summary)
        if (batch_idx + 1) % PRINT_FREQ == 0:
            utils.print_statistics(
                batch_idx, num_batches, epoch, MAX_EPOCH, batch_time, losses
            )
        end = time.time()

    if (epoch + 1) % EPOCH_EVAL_FREQ == 0 or epoch == MAX_EPOCH - 1:
        rank1, mAP = evaluate(model, test_loader)
        print(
            "Epoch {0}/{1}: Rank1: {rank}, mAP: {map}".format(
                epoch + 1, MAX_EPOCH, rank=rank1, map=mAP
            )
        )

## Save model

In [ ]:
model_path = reid_root_dir + "/models/resnet34_reid_market.model"
model_path

In [ ]:
torch.save(model.state_dict(), model_path)

## test load

In [ ]:
model = build_model(
    "resnet34", datamanager.num_train_pids, loss="triplet", pretrained=True
)
model = model.cuda()

In [ ]:
reid_market_state_dict = torch.load(
    model_path, map_location=lambda storage, loc: storage
)

In [ ]:
model.load_state_dict(reid_market_state_dict)

## eval model

In [ ]:
rank1, mAP = evaluate(model, test_loader)